# ETL Project

### Group Project by:

##### Bay Rodriguez  /  Chris Tran  /  Emil Rodulfo  /  Tim Besaw

In [1]:
# Dependencies and Setup
import pandas as pd
from sqlalchemy import create_engine
import sys

sys.path.append("static/assets/Resources/")
import config

## Extract Data

In [2]:
# Load and read game file
game_file = 'static/assets/Resources/game_stats.csv'
game_df = pd.read_csv(game_file, encoding='utf-8')

# Minimize data to bowl games from 2007-2016 and filter down fields pertinent to analysis
new_game_df = game_df.loc[game_df['Year'] >2006, :]
new_game_df = new_game_df[['Year', 'Away Team', 'Home Team', 'Bowl', 'id', 'Home Bowl Scores', 'Away Bowl Scores', 'Winning Team', 'Losing Team']]

new_game_df.head()

,Year,Away Team,Home Team,Bowl,id,Home Bowl Scores,Away Bowl Scores,Winning Team,Losing Team
1,2011,Washington Huskies,Baylor Bears,Alamo Bowl,17746,67,56,Baylor Bears,Washington Huskies
2,2016,Idaho Vandals,Colorado State Rams,Idaho Potato Bowl,27073,50,61,Idaho Vandals,Colorado State Rams
3,2015,Toledo Rockets,Arkansas State Red Wolves,GoDaddy Bowl,23972,44,63,Toledo Rockets,Arkansas State Red Wolves
4,2015,Tulsa Golden Hurricane,Virginia Tech Hokies,Independence Bowl,25533,55,52,Virginia Tech Hokies,Tulsa Golden Hurricane
5,2014,BYU Cougars,Memphis Tigers,Miami Beach Bowl,23939,55,48,Memphis Tigers,BYU Cougars


In [3]:
# Load and read player file
player_file = 'static/assets/Resources/player_stats.csv'
player_df = pd.read_csv(player_file, encoding='utf-8')

# Filter data to fields pertinent to analysis
new_player_df=player_df[['GameId','team','player']]

new_player_df.head()

,GameId,team,player
0,27067,UTSA Roadrunners,Sturm
1,27067,New Mexico Lobos,Jordan
2,27067,New Mexico Lobos,Apodaca
3,27067,UTSA Roadrunners,Williams
4,27067,UTSA Roadrunners,Rhodes


## Transform Data

In [4]:
# Rename game dataframe columns to match database field names
xform_game = new_game_df.rename(columns={'Year': 'year', 'Away Team': 'away_team', 'Home Team': 'home_team', 'Bowl': 'bowl', 'id': 'gameid', 'Home Bowl Scores': 'home_bowl_scores', 'Away Bowl Scores': 'away_bowl_scores', 'Winning Team': 'winning_team', 'Losing Team': 'losing_team'})

# Set index
xform_game.set_index('gameid', inplace=True)

xform_game.head()

,year,away_team,home_team,bowl,home_bowl_scores,away_bowl_scores,winning_team,losing_team
gameid,,,,,,,,
17746,2011,Washington Huskies,Baylor Bears,Alamo Bowl,67,56,Baylor Bears,Washington Huskies
27073,2016,Idaho Vandals,Colorado State Rams,Idaho Potato Bowl,50,61,Idaho Vandals,Colorado State Rams
23972,2015,Toledo Rockets,Arkansas State Red Wolves,GoDaddy Bowl,44,63,Toledo Rockets,Arkansas State Red Wolves
25533,2015,Tulsa Golden Hurricane,Virginia Tech Hokies,Independence Bowl,55,52,Virginia Tech Hokies,Tulsa Golden Hurricane
23939,2014,BYU Cougars,Memphis Tigers,Miami Beach Bowl,55,48,Memphis Tigers,BYU Cougars


In [5]:
# Rename player dataframe columns to match database field names
xform_player = new_player_df.rename(columns={'GameId':'gameid', 'team':'team', 'player':'player'})

# Set index
xform_player.set_index('gameid', inplace=True)

xform_player.head()

,team,player
gameid,,
27067,UTSA Roadrunners,Sturm
27067,New Mexico Lobos,Jordan
27067,New Mexico Lobos,Apodaca
27067,UTSA Roadrunners,Williams
27067,UTSA Roadrunners,Rhodes


## Load Data

In [6]:
# Create database connection
conn = (f'{config.username}:{config.password}@localhost/football_db')

engine = create_engine(f'mysql://{conn}')

ModuleNotFoundError: No module named 'MySQLdb'

In [ ]:
# Confirm tables
engine.table_names()

In [ ]:
# Load game dataframe to SQL database
xform_game.to_sql(name='game_stats', con=engine, if_exists = 'append', index=True)

In [ ]:
# Load players dataframe to SQL database
xform_player.to_sql(name='player_stats', con=engine, if_exists = 'append', index=True)